# CARE demo

This notebook covers the essential steps to generate and manipulate the chemical reaction networks (CRNs) generated with CARE. `care.scripts.care_run` is the recommended script to get the CRN, here we will cover the steps performed by the main script.
Install Ipykernel to run the notebook!

## Generate Chemical Reaction Network: Blueprint

To generate the CRN blueprint, i.e. the listing of the potential speciues and elementary reactions in your CRN, use the function `gen_blueprint`.

In [ ]:
from care.crn.utils.blueprint import gen_blueprint

### Inputs

*ncc* = Network Carbon Cutoff, i.e. the maximum number of C atoms that a species in the CRN can possess  <br>
*noc* = Network Oxygen Cutoff, i.e. the maximum number of C atoms that a species in the CRN can possess  <br>
*cyclic* = Whether species with rings are should be included or not (True/False)  <br>
*additional_rxns* = Whether to include intramolecular rearrengments (True/False)  <br>
*electro* = Whether to generate a CRN in electrochemical conditions

In [ ]:
ncc, noc = 2, 2
cyclic = False
rearr_rxns = True
electro = False

inters, rxns = gen_blueprint(ncc=ncc, 
                             noc=noc, 
                             cyclic=cyclic, 
                             additional_rxns=rearr_rxns, 
                             electro=electro)

The output of the function is a dict of `Intermediate` objects indexed by their InChIKey (plus a phase-identifier character, '*' for adsorbed, 'g' for gas-phase), and a list of `ElementaryReaction` instances.

In [ ]:
print(type(inters)) 
print(type(inters[list(inters.keys())[0]]))
print(type(rxns))
print(type(rxns[0]))

### ElementaryReaction

In [ ]:
r = rxns[10]
print(r)
print(type(r))
print(r.repr_hr)  # human-readable text representation
print(r.r_type)  # bond-breaking type
print(r.components)  # components of the reaction
print(r.stoic)  # Soichiometry coefficients in the reaction
print(r.e_rxn)  # reaction energy
print(r.e_act)  # reaction activation energy

Observations:
- species participating in the elementary reactions are displayed as '[n]InChIKeyP(formula)', with 'n' being the absolute stoichiometric coefficient, formula being the brute formula provided by ASE, and 'P' after the InChIKey being a phase-identifier character, '*' for adsorbed species and 'g' for gaseous species. <br>
- ``e_rxn`` and ``e_act`` are ``None`` as these reaction properties still have to be evaluated


### Intermediate

In [ ]:
a = inters['WSFSSNUMVMOOMR-UHFFFAOYSA-N*']
print(a)
print(type(a))
print(a.phase)
print(a.smiles)
print(a.ref_energy())
print(a.is_closed_shell())
print(a['C'])  # number of carbon atoms
print(a['H'])  # number of hydrogen atoms
print(a['O'])  # number of oxygen atoms
print(a.ads_configs)  # adsorption configurations

Observations:

## Generate Chemical Reaction Network: Energy Evaluation

Now we define the surface under study and evaluate the energies of intermediates and reactions

## Load catalyst surface

In [ ]:
from care.utils import load_surface
from care import DB_PATH

In [ ]:
metal = 'Ru'
facet = '0001'  # hkl notation

surface = load_surface(DB_PATH, metal, facet)

In [ ]:
print(surface)
print(type(surface))
print(surface.slab)  # ASE Atoms object
print(surface.active_sites)  # Obtained with ACAT

## Load Intermediate energy evaluator

In our case, this is GAME-Net-UQ, a graph neural network trained on DFT data.

In [ ]:
from care.gnn.interface import GameNetUQInter
from care import MODEL_PATH

In [ ]:
gnn = GameNetUQInter(MODEL_PATH, surface, None)  # None for the ase db (will be available later)

## Evaluate intermediates (Sequential)

We have 171 intermediates to evaluate. We will show here the sequential evaluation to give an idea. If you want to get the same output faster, skip this section and run the parallel evaluation.
The sequential evaluation takes some time for this case, in case you do not want to wait, reduce ncc and noc. 

In [ ]:
inters_evaluated = {}
for k, inter in inters.items():
    print(inter)
    inters_evaluated[k] = gnn.eval(inter)
    print(inters_evaluated[k].ads_configs)
    

In [ ]:
inters_evaluated['LHGAACIDYUKUTF-UHFFFAOYSA-N*'].ads_configs

## Evaluate intermediates (parallel execution)

As the sequential execution could take some time, here we show how parallel execution across multiple CPU core can speed up the evaluation process. In our case, the parallel execution took 1min 51s, while the sequential 20 minutes (10x speed up with 24 cores).

In [ ]:
import multiprocessing as mp
import resource

_, hard = resource.getrlimit(resource.RLIMIT_NOFILE)
resource.setrlimit(resource.RLIMIT_NOFILE, (hard, hard))

In [ ]:
gnn = GameNetUQInter(MODEL_PATH, surface, None)

tasks = [(gnn, inter) for inter in inters.values()]
print(f'{len(tasks)} intermediates to evaluate')

In [ ]:
def mp_func(gnn, inter):
    print(inter.code + ' \n')
    return gnn.eval(inter)

In [ ]:
with mp.Pool(mp.cpu_count()) as pool:
    results = pool.starmap(mp_func, tasks)

inters_evaluated = {i.code: i for i in results}

## Evaluate reaction properties

In [ ]:
from care.gnn.interface import GameNetUQRxn

In [ ]:
gnn_rxn = GameNetUQRxn(MODEL_PATH, inters)
print(gnn_rxn)

print(f'{len(rxns)} reactions to evaluate')

In [ ]:
rxns_evaluated = []
for rxn in rxns:
    print(rxn)
    rxns_evaluated.append(gnn_rxn.eval(rxn))

rxns_evaluated = sorted(rxns_evaluated)

In [ ]:
import random
idx = random.randint(0, len(rxns_evaluated))

step_example = rxns_evaluated[idx]
print(step_example, step_example.r_type)
print(step_example.e_rxn)
print(step_example.e_act)

## Explore the CRN

Everything is now evaluated. Let's assemble the reaction network.


In [ ]:
from care import ReactionNetwork

In [ ]:
crn = ReactionNetwork(intermediates=inters_evaluated, 
                      reactions=rxns_evaluated,
                      surface=surface,
                      ncc=ncc,
                        noc=noc,
                      type='thermal')

print(crn)

Get top-10 hubs in the network

In [ ]:
crn.get_hubs(10)

Iterate through elementary steps

In [ ]:
for step in crn:
    print(step.repr_hr, step.e_rxn, step.e_act)

Visualize elementary steps

In [ ]:
crn.visualize_reaction(42, show_uncertainty=True)

Get number of closed-shell molecules in the network

In [ ]:
crn.num_closed_shell_mols

Visualize intermediates

In [ ]:
crn.visualize_intermediate('VRLIPUYDFBXWCH-UHFFFAOYSA-N*')

Visualize graph representation

In [ ]:
from care.gnn.graph_tools import graph_plotter

# Soon! Need to change a function!

inter = crn.intermediates['VRLIPUYDFBXWCH-UHFFFAOYSA-N*']
inter.ads_configs['21']['pyg']